# LangChain Agent with Qiskit MCP Server

This notebook demonstrates how to create an AI agent using LangChain that connects to the **qiskit-mcp-server** via the Model Context Protocol (MCP).

The agent can interact with Qiskit's transpiler to:
- Transpile quantum circuits with configurable optimization levels (0-3)
- Analyze circuit structure and complexity
- Compare optimization levels to find the best settings

## Architecture

```
┌─────────────┐     MCP Protocol     ┌──────────────────────────┐
│  LangChain  │ ◄──────────────────► │    qiskit-mcp-server     │
│    Agent    │                      │                          │
└─────────────┘                      │  ┌────────────────────┐  │
                                     │  │  Qiskit Transpiler │  │
                                     │  └────────────────────┘  │
                                     └──────────────────────────┘
```

## Setup

### 1. Install Dependencies

Run these commands in your terminal:

```bash
# Install the MCP server
pip install qiskit-mcp-server

# Install LangChain dependencies
pip install langchain langchain-mcp-adapters python-dotenv

# Install your preferred LLM provider (choose one):
pip install langchain-openai       # For OpenAI
pip install langchain-anthropic    # For Anthropic Claude
pip install langchain-google-genai # For Google Gemini
pip install langchain-ollama       # For local Ollama
pip install langchain-ibm          # For IBM Watsonx
```

### 2. Configure Environment Variables

Set your LLM provider API key.

You can either:
- Set them in a `.env` file in this directory
- Set them as environment variables
- Enter them in the cell below

**Note:** This server doesn't require IBM Quantum credentials - it uses local Qiskit transpilation.

In [1]:
import os

from dotenv import load_dotenv

# LangChain imports
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_mcp_adapters.tools import load_mcp_tools


# Load from .env file if it exists
load_dotenv()

# Set your LLM provider API key (uncomment the one you're using):
# os.environ["OPENAI_API_KEY"] = "your-openai-api-key"
# os.environ["ANTHROPIC_API_KEY"] = "your-anthropic-api-key"
# os.environ["GOOGLE_API_KEY"] = "your-google-api-key"

# Verify configuration
print("Configuration status:")
print(f"  OPENAI_API_KEY: {'✓ Set' if os.getenv('OPENAI_API_KEY') else '✗ Not set'}")
print(f"  ANTHROPIC_API_KEY: {'✓ Set' if os.getenv('ANTHROPIC_API_KEY') else '✗ Not set'}")

Configuration status:
  OPENAI_API_KEY: ✗ Not set
  ANTHROPIC_API_KEY: ✓ Set


## Choose Your LLM Provider

Run **one** of the following cells based on your preferred LLM provider:

In [ ]:
# Option 1: OpenAI
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(model="gpt-4o", temperature=0)
print("Using OpenAI GPT-4o")

In [ ]:
# Option 2: Anthropic Claude
from langchain_anthropic import ChatAnthropic


llm = ChatAnthropic(model="claude-sonnet-4-20250514", temperature=0)
print("Using Anthropic Claude Sonnet")

Using Anthropic Claude Sonnet


In [ ]:
# Option 3: Google Gemini
from langchain_google_genai import ChatGoogleGenerativeAI


llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro", temperature=0)
print("Using Google Gemini Pro")

In [ ]:
# Option 4: Local Ollama (no API key needed)
from langchain_ollama import ChatOllama


llm = ChatOllama(model="llama3.2", temperature=0)
print("Using local Ollama with Llama 3.2")

In [ ]:
# Option 5: IBM Watsonx
from langchain_ibm import ChatWatsonx


llm = ChatWatsonx(
    model_id="ibm/granite-3-8b-instruct",
    url=os.getenv("WATSONX_URL", "https://us-south.ml.cloud.ibm.com"),
    project_id=os.getenv("WATSONX_PROJECT_ID"),
    params={"temperature": 0, "max_tokens": 4096},
)
print("Using IBM Watsonx Granite")

## Define the System Prompt

This prompt tells the agent what it can do and how to behave:

In [5]:
SYSTEM_PROMPT = """You are a helpful quantum computing assistant with access to Qiskit's quantum circuit
transpiler through the MCP server.

You can help users optimize their quantum circuits using:
- transpile_circuit_tool: Transpile circuits with configurable optimization levels (0-3)
- analyze_circuit_tool: Analyze circuit structure without transpiling
- compare_optimization_levels_tool: Compare all optimization levels (0-3)

When working with circuits:
1. Accept QASM 3.0 or QASM 2.0 circuit strings from users
2. Use the appropriate tool based on what the user needs
3. Explain the results clearly, including gate counts, depth, and optimizations

Available basis gate presets:
- ibm_eagle: IBM Eagle r3 processors (id, rz, sx, x, ecr, reset)
- ibm_heron: IBM Heron processors (id, rz, sx, x, cz, reset)
- ibm_legacy: Older IBM systems (id, rz, sx, x, cx, reset)

Available topologies:
- linear: Chain connectivity (qubit i <-> i+1)
- ring: Linear with wraparound
- grid: 2D grid connectivity
- heavy_hex: IBM heavy-hex topology
- full: All-to-all connectivity

Optimization levels:
- 0: No optimization, only basis gate decomposition (fastest)
- 1: Light optimization with default layout
- 2: Medium optimization with noise-aware layout (recommended)
- 3: Heavy optimization for best results (can be slow for large circuits)

Always explain the trade-offs between optimization levels and help users choose the best one.
"""

## Define Sample Circuits

Here are some sample QASM circuits for testing:

In [6]:
# Sample Bell state circuit
SAMPLE_BELL_STATE = """OPENQASM 3.0;
include "stdgates.inc";
qubit[2] q;
h q[0];
cx q[0], q[1];
"""

# Sample GHZ state circuit
SAMPLE_GHZ_STATE = """OPENQASM 3.0;
include "stdgates.inc";
qubit[4] q;
h q[0];
cx q[0], q[1];
cx q[1], q[2];
cx q[2], q[3];
"""

# Sample QFT circuit
SAMPLE_QFT_CIRCUIT = """OPENQASM 3.0;
include "stdgates.inc";
qubit[3] q;
h q[0];
cp(pi/2) q[1], q[0];
cp(pi/4) q[2], q[0];
h q[1];
cp(pi/2) q[2], q[1];
h q[2];
swap q[0], q[2];
"""

print("Sample circuits defined!")
print("\nBell state circuit:")
print(SAMPLE_BELL_STATE)
print("\nGHZ state circuit:")
print(SAMPLE_GHZ_STATE)
print("\nQFT circuit:")
print(SAMPLE_QFT_CIRCUIT)

Sample circuits defined!

Bell state circuit:
OPENQASM 3.0;
include "stdgates.inc";
qubit[2] q;
h q[0];
cx q[0], q[1];


GHZ state circuit:
OPENQASM 3.0;
include "stdgates.inc";
qubit[4] q;
h q[0];
cx q[0], q[1];
cx q[1], q[2];
cx q[2], q[3];


QFT circuit:
OPENQASM 3.0;
include "stdgates.inc";
qubit[3] q;
h q[0];
cp(pi/2) q[1], q[0];
cp(pi/4) q[2], q[0];
h q[1];
cp(pi/2) q[2], q[1];
h q[2];
swap q[0], q[2];



## Create the MCP Client

This configures the connection to the qiskit-mcp-server:

In [7]:
def get_mcp_client():
    """Create and return an MCP client configured for the Qiskit MCP server."""
    return MultiServerMCPClient(
        {
            "qiskit": {
                "transport": "stdio",
                "command": "qiskit-mcp-server",
                "args": [],
                "env": {},
            }
        }
    )

## Create the Agent

Now we'll create a function that sets up the agent with a persistent MCP session.

Using a persistent session is important because it:
- Keeps a single MCP server process running
- Makes tool calls much faster

In [8]:
async def create_agent_with_session(session):
    """Create a LangChain agent using an existing MCP session."""
    # Load tools from the existing session
    tools = await load_mcp_tools(session)
    print(f"Loaded {len(tools)} tools from MCP server:")
    for tool in tools:
        print(f"  - {tool.name}")

    # Create the agent using LangChain's create_agent
    agent = create_agent(llm, tools, system_prompt=SYSTEM_PROMPT)
    return agent

## Helper Function to Run Queries

This function sends a query to the agent and returns the response:

In [9]:
async def ask_agent(agent, query: str) -> str:
    """Send a query to the agent and return the response."""
    result = await agent.ainvoke({"messages": [HumanMessage(content=query)]})
    messages = result.get("messages", [])
    if messages:
        return messages[-1].content
    return "No response generated."

## Run the Agent

Now let's create the agent and ask it some questions!

The following cell starts the MCP server, creates the agent, and keeps the session open for multiple queries:

In [10]:
# Create MCP client and start a persistent session
mcp_client = get_mcp_client()

print("Starting MCP server and creating agent...")
print("(This may take a few seconds on first run)\n")

Starting MCP server and creating agent...
(This may take a few seconds on first run)



### Example 1: Analyze a Bell State Circuit

Let's ask the agent to analyze a simple Bell state circuit:

In [11]:
async with mcp_client.session("qiskit") as session:
    agent = await create_agent_with_session(session)

    query = f"""Analyze this Bell state circuit and tell me about its structure:

{SAMPLE_BELL_STATE}"""

    response = await ask_agent(agent, query)
    print(response)

Loaded 5 tools from MCP server:
  - transpile_circuit_tool
  - analyze_circuit_tool
  - compare_optimization_levels_tool
  - convert_qpy_to_qasm3_tool
  - convert_qasm3_to_qpy_tool
Perfect! Here's a detailed analysis of your Bell state circuit:

## Circuit Structure

**Basic Properties:**
- **Number of Qubits:** 2
- **Circuit Depth:** 2 (layers of gates)
- **Total Operations:** 2 gates
- **Classical Bits:** 0 (no measurements)

## Gate Breakdown

| Gate Type | Count |
|-----------|-------|
| Hadamard (H) | 1 |
| CNOT (CX) | 1 |

**Gate Categories:**
- Single-qubit gates: 1 (the Hadamard)
- Two-qubit gates: 1 (the CNOT)

## What This Circuit Does

This is the classic **Bell state preparation circuit** that creates a maximally entangled state (specifically, the |Φ⁺⟩ Bell state):

1. **H q[0]**: Applies a Hadamard gate to qubit 0, putting it into a superposition: (|0⟩ + |1⟩)/√2
2. **CX q[0], q[1]**: Applies a CNOT with qubit 0 as control and qubit 1 as target, creating entanglement

**Fin

### Example 2: Transpile for IBM Heron

Transpile a circuit for IBM's Heron processor:

In [12]:
async with mcp_client.session("qiskit") as session:
    agent = await create_agent_with_session(session)

    query = f"""Transpile this GHZ circuit for IBM Heron with optimization level 2:

{SAMPLE_GHZ_STATE}

Explain what changes were made during transpilation."""

    response = await ask_agent(agent, query)
    print(response)

Loaded 5 tools from MCP server:
  - transpile_circuit_tool
  - analyze_circuit_tool
  - compare_optimization_levels_tool
  - convert_qpy_to_qasm3_tool
  - convert_qasm3_to_qpy_tool
Let me provide you with a detailed analysis of the transpilation results:

## Transpilation Summary

Your GHZ circuit has been successfully transpiled for IBM Heron with optimization level 2. Here's what changed:

### **Original Circuit:**
- **Depth:** 4
- **Gate count:** 4 gates total
  - 1 Hadamard (H)
  - 3 CNOT (CX) gates
- **Basis gates:** H, CX

### **Transpiled Circuit:**
- **Depth:** 12
- **Gate count:** 21 gates total
  - 11 RZ gates
  - 7 SX gates
  - 3 CZ gates
- **Basis gates:** RZ, SX, CZ (IBM Heron native gates)

### **Key Changes Made:**

1. **CNOT → CZ Decomposition**: The three CNOT gates were converted to CZ gates, which are the native two-qubit gates on IBM Heron. Each CNOT requires decomposition into SX and CZ gates.

2. **Hadamard → SX + RZ**: The Hadamard gate was decomposed into SX and

### Example 3: Compare Optimization Levels

Compare all optimization levels for a QFT circuit:

In [13]:
async with mcp_client.session("qiskit") as session:
    agent = await create_agent_with_session(session)

    query = f"""Compare optimization levels 0-3 for this QFT circuit and recommend the best one:

{SAMPLE_QFT_CIRCUIT}"""

    response = await ask_agent(agent, query)
    print(response)

Loaded 5 tools from MCP server:
  - transpile_circuit_tool
  - analyze_circuit_tool
  - compare_optimization_levels_tool
  - convert_qpy_to_qasm3_tool
  - convert_qasm3_to_qpy_tool
Excellent! Here's a detailed comparison of all optimization levels for your QFT circuit:

## Optimization Results Summary

| Metric | Original | Level 0 | Level 1 | Level 2 | Level 3 |
|--------|----------|---------|---------|---------|---------|
| **Depth** | 6 | 6 | 6 | **5** ✓ | **5** ✓ |
| **Size** | 7 | 7 | 7 | **6** ✓ | **6** ✓ |
| **H Gates** | 3 | 3 | 3 | 3 | 3 |
| **CP Gates** | 3 | 3 | 3 | 3 | 3 |
| **SWAP Gates** | 1 | 1 | 1 | 0 | 0 |

## Key Findings

**Levels 0 & 1:** No optimization applied
- Keep the original circuit structure unchanged
- Fastest compilation but no improvements

**Levels 2 & 3:** Significant improvements
- **Removed the SWAP gate** (1 operation eliminated)
- **Reduced circuit depth by 1** (from 6 to 5)
- Both achieve identical results for this circuit

## Recommendation: **Lev

### Example 4: Transpile with Linear Topology

Transpile a circuit with a specific coupling map:

In [14]:
async with mcp_client.session("qiskit") as session:
    agent = await create_agent_with_session(session)

    query = f"""Transpile this GHZ circuit with a linear topology (chain connectivity):

{SAMPLE_GHZ_STATE}

How many SWAP gates were added?"""

    response = await ask_agent(agent, query)
    print(response)

Loaded 5 tools from MCP server:
  - transpile_circuit_tool
  - analyze_circuit_tool
  - compare_optimization_levels_tool
  - convert_qpy_to_qasm3_tool
  - convert_qasm3_to_qpy_tool
Perfect! Here are the results:

## Transpilation Results

**Good news: No SWAP gates were added!** ✓

Your GHZ circuit is already perfectly compatible with linear (chain) topology. Here's why:

### Circuit Analysis:
- **Original circuit**: 4 gates (1 H + 3 CX)
- **Transpiled circuit**: 4 gates (1 H + 3 CX)
- **Depth**: 4 (unchanged)
- **SWAP gates added**: **0**

### Why No SWAPs Were Needed:
Your circuit has a natural chain structure:
- `H q[0]` - operates on qubit 0
- `CX q[0], q[1]` - connects adjacent qubits (0→1) ✓
- `CX q[1], q[2]` - connects adjacent qubits (1→2) ✓
- `CX q[2], q[3]` - connects adjacent qubits (2→3) ✓

Since all your two-qubit gates operate on adjacent qubits in the linear topology, the circuit requires no routing overhead. This is an ideal case for chain connectivity!


### Example 5: Interactive Chat

Run this cell to have an interactive conversation with the agent:

In [15]:
async with mcp_client.session("qiskit") as session:
    agent = await create_agent_with_session(session)
    print("Agent ready! Type your questions below.")
    print("Enter 'quit' to stop.\n")

    while True:
        try:
            query = input("You: ").strip()
            if not query:
                continue
            if query.lower() in ["quit", "exit", "q"]:
                print("Goodbye!")
                break

            response = await ask_agent(agent, query)
            print(f"\nAssistant: {response}\n")
        except KeyboardInterrupt:
            print("\nGoodbye!")
            break

Loaded 5 tools from MCP server:
  - transpile_circuit_tool
  - analyze_circuit_tool
  - compare_optimization_levels_tool
  - convert_qpy_to_qasm3_tool
  - convert_qasm3_to_qpy_tool
Agent ready! Type your questions below.
Enter 'quit' to stop.


Assistant: Hi! 👋 Welcome! I'm your quantum computing assistant, and I'm here to help you optimize and analyze quantum circuits using Qiskit's transpiler.

Here's what I can help you with:

**🔧 Transpile Circuits**
- Optimize your quantum circuits for specific hardware
- Choose optimization levels (0-3) based on your needs
- Target different basis gates (IBM Eagle, Heron, etc.)
- Specify qubit connectivity (linear, ring, grid, etc.)

**📊 Analyze Circuits**
- Get detailed information about circuit structure
- View gate counts and circuit depth
- Understand circuit complexity

**⚖️ Compare Optimization Levels**
- See how different optimization levels affect your circuit
- Make informed decisions about speed vs. quality trade-offs

**How to get star

## Custom Queries

Use this cell to transpile your own circuits:

In [16]:
# Enter your QASM circuit here:
MY_CIRCUIT = """OPENQASM 3.0;
include "stdgates.inc";
qubit[3] q;
h q[0];
cx q[0], q[1];
cx q[1], q[2];
"""

MY_REQUEST = "Transpile this circuit for IBM Eagle with optimization level 3"

async with mcp_client.session("qiskit") as session:
    agent = await create_agent_with_session(session)

    query = f"""{MY_REQUEST}:

{MY_CIRCUIT}"""

    response = await ask_agent(agent, query)
    print(response)

Loaded 5 tools from MCP server:
  - transpile_circuit_tool
  - analyze_circuit_tool
  - compare_optimization_levels_tool
  - convert_qpy_to_qasm3_tool
  - convert_qasm3_to_qpy_tool
Perfect! Your circuit has been transpiled for IBM Eagle with optimization level 3. Here's a summary:

## Transpilation Results

**Original Circuit:**
- Qubits: 3
- Depth: 3
- Size: 3 gates
- Operations: 1 H gate, 2 CX gates

**Transpiled Circuit:**
- Qubits: 3
- Depth: 7
- Size: 13 gates
- Operations: 6 RZ gates, 3 SX gates, 2 ECR gates, 2 X gates

## Key Points

The transpiled circuit uses IBM Eagle's native basis gates:
- **RZ** (rotation around Z-axis)
- **SX** (√X gate)
- **ECR** (echoed cross-resonance, Eagle's native 2-qubit gate)
- **X** (Pauli X)

The increase in depth and gate count is expected because:
1. **CX gates are decomposed**: IBM Eagle doesn't have native CX gates, so they're decomposed into ECR gates with additional single-qubit rotations
2. **H gates are decomposed**: H gates are converte

## Available Tools

The agent has access to these tools provided by the MCP server:

| Tool | Description |
|------|-------------|
| `transpile_circuit_tool` | Transpile a circuit with configurable optimization |
| `analyze_circuit_tool` | Analyze circuit structure without transpiling |
| `compare_optimization_levels_tool` | Compare all optimization levels (0-3) |

### Basis Gate Presets

| Preset | Gates | Description |
|--------|-------|-------------|
| `ibm_eagle` | id, rz, sx, x, ecr, reset | IBM Eagle r3 (127 qubits, uses ECR) |
| `ibm_heron` | id, rz, sx, x, cz, reset | IBM Heron (133-156 qubits, uses CZ) |
| `ibm_legacy` | id, rz, sx, x, cx, reset | Older IBM systems (uses CX) |

### Available Topologies

| Topology | Description |
|----------|-------------|
| `linear` | Chain connectivity (qubit i ↔ i+1) |
| `ring` | Linear with wraparound |
| `grid` | 2D grid connectivity |
| `heavy_hex` | IBM heavy-hex topology |
| `full` | All-to-all connectivity |

## Troubleshooting

### MCP server not found?
Make sure `qiskit-mcp-server` is installed: `pip install qiskit-mcp-server`

### Invalid QASM?
- Ensure your QASM circuit is valid QASM 3.0 or QASM 2.0 syntax
- Include the `include "stdgates.inc";` line for standard gates

### Transpilation slow?
- Use optimization level 2 instead of 3 for larger circuits
- Level 3 can be very slow for circuits with >20 qubits or >500 gates